In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometrioid_Cancer"
cohort = "GSE66667"

# Input paths
in_trait_dir = "../../input/GEO/Endometrioid_Cancer"
in_cohort_dir = "../../input/GEO/Endometrioid_Cancer/GSE66667"

# Output paths
out_data_file = "../../output/preprocess/Endometrioid_Cancer/GSE66667.csv"
out_gene_data_file = "../../output/preprocess/Endometrioid_Cancer/gene_data/GSE66667.csv"
out_clinical_data_file = "../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE66667.csv"
json_path = "../../output/preprocess/Endometrioid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Tumorgrafts as in vivo surrogates for women with ovarian cancer."
!Series_summary	"Purpose:Ovarian cancer has a high recurrence and mortality rate. A barrier to improved outcomes includes a lack of accurate models for preclinical testing of novel therapeutics. "
!Series_summary	"Experimental Design:Clinically-relevant, patient-derived tumorgraft models were generated from sequential patients and the first 168 engrafted models are described. Fresh ovarian, primary peritoneal, and fallopian tube carcinomas were collected at the time of debulking surgery and injected intraperitoneally into severe combined immunodeficient mice. "
!Series_summary	"Results:Tumorgrafts demonstrated a 74% engraftment rate with microscopic fidelity of primary tumor characteristics. Low-passage tumorgrafts also showed comparable genomic aberrations with the corresponding primary tumor and exhibit gene set enrichment of multiple ovarian cancer molecular subtypes, similar to 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Looking at the background information, the study mentions microarrays for global transcription
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Endometrioid Cancer), we can identify this from histology information (row 0)
trait_row = 0

# For age and gender, they don't appear to be available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert histology information to binary indicator for Endometrioid Cancer
    1 means Endometrioid Cancer, 0 means not Endometrioid Cancer
    """
    if pd.isna(value):
        return None
    # Extract value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the histology is Endometrioid
    if value.lower() == 'endometrioid':
        return 1
    else:
        return 0

# No age data available
convert_age = None

# No gender data available
convert_gender = None

# 3. Save Metadata
# Conduct initial filtering
is_trait_available = trait_row is not None
result = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # The clinical_data must have been parsed in a previous step
    # Use the Sample Characteristics Dictionary for clinical data
    sample_chars = {
        0: ['histology: Carcinosarcoma', 'histology: Serous', 'histology: Endometrioid', 
            'histology: Mixed', 'histology: Undifferentiated', 'histology: Clear Cell'],
        1: ['Stage: IIIC', 'Stage: IIA', 'Stage: IIC', 'Stage: IV', 'Stage: IC', 
            'Stage: Unstaged', 'Stage: IIIB'],
        2: ['surgery timing: Primary', 'surgery timing: NA'],
        3: ['debulking category: Optimal', 'debulking category: NA', 'debulking category: Sub-optimal'],
        4: ['primary treatment: Chemo', 'primary treatment: NA'],
        5: ['platinum: Yes', 'platinum: NA', 'platinum: No'],
        6: ['taxane: Yes', 'taxane: NA', 'taxane: No'],
        7: ['other chemo: No', 'other chemo: NA', 'other chemo: Yes'],
        8: ['vital status: Dead', 'vital status: Alive']
    }
    
    # Convert the sample characteristics to a DataFrame format
    clinical_data = pd.DataFrame.from_dict(sample_chars, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0], 1: [0.0], 2: [1.0], 3: [0.0], 4: [0.0], 5: [0.0], 6: [nan]}
Clinical data saved to ../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE66667.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 74
Header line: "ID_REF"	"GSM1627456"	"GSM1627457"	"GSM1627458"	"GSM1627459"	"GSM1627460"	"GSM1627461"	"GSM1627462"	"GSM1627463"	"GSM1627464"	"GSM1627465"	"GSM1627466"	"GSM1627467"	"GSM1627468"	"GSM1627469"	"GSM1627470"	"GSM1627471"	"GSM1627472"	"GSM1627473"	"GSM1627474"	"GSM1627475"	"GSM1627476"	"GSM1627477"	"GSM1627478"	"GSM1627479"	"GSM1627480"	"GSM1627481"	"GSM1627482"	"GSM1627483"	"GSM1627484"	"GSM1627485"	"GSM1627486"	"GSM1627487"	"GSM1627488"	"GSM1627489"	"GSM1627490"	"GSM1627491"
First data line: "1007_s_at"	8.518574742	10.1246373	9.572614398	9.957276346	9.246003592	9.446475616	9.51311659	7.283029341	9.700293986	9.909452113	8.706882287	10.13979277	10.29675592	8.878101648	9.857782641	9.8906665	10.86173282	10.21935341	9.883848624	9.164688065	9.201274305	7.386178388	10.13235351	10.12564917	8.66662704	9.685041964	9.666215337	9.542360393	9.485895918	9.97491352	9.626821561	10.05933893	9.443030292	9.487844181	10.20489702	10.1618931


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the output
# The identifiers like "1007_s_at", "1053_at", etc. appear to be probe IDs from a microarray platform
# These are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE66667
Line 6: !Series_title = Tumorgrafts as in vivo surrogates for women with ovarian cancer.
Line 7: !Series_geo_accession = GSE66667
Line 8: !Series_status = Public on Mar 09 2015
Line 9: !Series_submission_date = Mar 09 2015
Line 10: !Series_last_update_date = Mar 25 2019
Line 11: !Series_pubmed_id = 24398046
Line 12: !Series_summary = Purpose:Ovarian cancer has a high recurrence and mortality rate. A barrier to improved outcomes includes a lack of accurate models for preclinical testing of novel therapeutics.
Line 13: !Series_summary = Experimental Design:Clinically-relevant, patient-derived tumorgraft models were generated from sequential patients and the first 168 engrafted models are described. F


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which columns to use for gene mapping
# From previous steps' outputs, we can see:
# - The gene expression data uses identifiers like "1007_s_at", stored in the index
# - The gene annotation data has these identifiers in the 'ID' column
# - Gene symbols are stored in the 'Gene Symbol' column

# 2. Extract the mapping between probe IDs and gene symbols
mapping_df = gene_annotation.loc[:, ['ID', 'Gene Symbol']].copy()
mapping_df = mapping_df.rename(columns={'Gene Symbol': 'Gene'})

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the mapped gene expression data
print("\nPreview of gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")
print(f"First 5 columns: {gene_data.columns[:5].tolist()}")

# Save the gene expression data to a CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")



Preview of gene expression data after mapping:
Shape: (21278, 36)
First 5 gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1']
First 5 columns: ['GSM1627456', 'GSM1627457', 'GSM1627458', 'GSM1627459', 'GSM1627460']


Gene expression data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE66667.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (19845, 36)
First few genes with their expression values after normalization:
          GSM1627456  GSM1627457  GSM1627458  GSM1627459  GSM1627460  \
Gene                                                                   
A1BG        6.253131    6.699967    6.927761    5.681226    5.914132   
A1BG-AS1    5.028280    5.142778    5.265763    4.470213    4.629661   
A1CF        8.456704    7.464496    7.870192    7.761571    7.984048   
A2M         9.850656   15.797511    9.394025   10.832497    8.886093   
A2M-AS1     3.924116    7.436648    3.929899    5.086617    3.732718   

          GSM1627461  GSM1627462  GSM1627463  GSM1627464  GSM1627465  ...  \
Gene                                                                  ...   
A1BG        6.058819    5.173376    7.955459    5.561192    7.139684  ...   
A1BG-AS1    4.860277    4.906534    5.799538    4.882458    5.169499  ...   
A1CF        8.408178    8.022555    7.980765    7.684633    7.930522  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE66667.csv
Raw clinical data shape: (9, 37)
Clinical features:
                     GSM1627456  GSM1627457  GSM1627458  GSM1627459  \
Endometrioid_Cancer         0.0         0.0         0.0         1.0   

                     GSM1627460  GSM1627461  GSM1627462  GSM1627463  \
Endometrioid_Cancer         0.0         0.0         0.0         0.0   

                     GSM1627464  GSM1627465  ...  GSM1627482  GSM1627483  \
Endometrioid_Cancer         0.0         0.0  ...         0.0         0.0   

                     GSM1627484  GSM1627485  GSM1627486  GSM1627487  \
Endometrioid_Cancer         0.0         0.0         0.0         0.0   

                     GSM1627488  GSM1627489  GSM1627490  GSM1627491  
Endometrioid_Cancer         0.0         0.0         0.0         0.0  

[1 rows x 36 columns]
Clinical features saved to ../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE66667.csv
Linked d

Data shape after handling missing values: (36, 19846)
For the feature 'Endometrioid_Cancer', the least common label is '1.0' with 1 occurrences. This represents 2.78% of the dataset.
The distribution of the feature 'Endometrioid_Cancer' in this dataset is severely biased.

Data was determined to be unusable or empty and was not saved
